In [1]:
### JUPYTER NOTEBOOK / MODULE PATH FIX ###
import sys

if "/share/apps/python/3.8.6/intel/lib/python3.8" in sys.path and sys.path[1] == '':
    sys.path[1], sys.path[6] = sys.path[6], sys.path[1]

In [2]:
from jtb_2022_code.utils.figure_common import *
from jtb_2022_code.utils.adata_common import *
from jtb_2022_code.utils.pseudotime_common import *
from jtb_2022_code import FigureSingleCellData
from jtb_2022_code.utils.decay_common import calc_decays, calc_halflives
from inferelator_prior.velocity import decay
import numpy as np
import pandas as pd

In [4]:
## LOAD AND PROCESS DATA ###
data = FigureSingleCellData()
data.load_published_decay()

#def _calc_decay_for_untreated(adata, *args, **kwargs):
#    untreated = adata[(adata.obs['Pool'] == 1) | (adata.obs['Pool'] == 2), :].copy()
#    calc_decays(untreated, *args, decay_key='unperturbed_decay', alpha_key='unperturbed_alpha')
#    adata.var['unperturbed_decay'] = untreated.var['unperturbed_decay'].copy()
    
#    return adata
    
#data.apply_inplace_to_expts(_calc_decay_for_untreated)

Reading Single Cell Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_1_WT.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_1_fpr1.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_2_WT.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_2_fpr1.h5ad


/home/cj59/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
from scipy.stats import spearmanr
import itertools
import pandas as pd

half_lives = [("Expt. 1", data.expt_data[(1, "WT")].var['unperturbed_decay']),
              ("Expt. 2", data.expt_data[(2, "WT")].var['unperturbed_decay']),
              ("Neymotin2014", data.all_data.var['Neymotin2014']),
              ("Chan2018", data.all_data.var['Chan2018']),
              ("Munchel2011", data.all_data.var['Munchel2011']),
              ("Miller2011", data.all_data.var['Miller2011']),
              ("Geisberg2015", data.all_data.var['Geisberg2015'])]

hl_spearmans = pd.DataFrame([[x[0], y[0], spearmanr(x[1], y[1], nan_policy='omit')[0]] 
                             for x, y in itertools.product(half_lives, half_lives)])

In [7]:
hl_spearmans

,0,1,2
0,Expt. 1,Expt. 1,1.000000
1,Expt. 1,Expt. 2,0.930454
2,Expt. 1,Neymotin2014,-0.109802
3,Expt. 1,Chan2018,-0.427838
4,Expt. 1,Munchel2011,-0.083788
5,Expt. 1,Miller2011,-0.505319
6,Expt. 1,Geisberg2015,0.136677
7,Expt. 2,Expt. 1,0.930454
8,Expt. 2,Expt. 2,1.000000
9,Expt. 2,Neymotin2014,-0.160630
